## owyl-2

- 阅读 owyl 源码

In [6]:
def wrap(func, *args, **kwargs):
    """Wrap a callable as a task. Yield the boolean of its result.
    """
    def initTask(**initkwargs):
        def makeIterator(**runkwargs):
            result = func(*args, **kwargs)
            yield bool(result)
        try: makeIterator.__name__ = func.__name__
        except AttributeError: pass
        try: makeIterator.__doc__ = func.__doc__
        except AttributeError: pass
        return makeIterator
    try: initTask.__doc__ = func.__doc__
    except AttributeError: pass
    try: initTask.__name__ = func.__name__
    except AttributeError: pass
    return initTask

In [7]:
@wrap
def func():
    print 1

In [9]:
f = func()

In [11]:
f

<function __main__.func>

In [13]:
c = f()

In [14]:
c

<generator object makeIterator at 0x05E5E918>

In [15]:
next(c)

1


False

In [16]:
def dwrap(func, *args, **kwargs):
    """Wrap a callable as a task. Yield the boolean of its result.
    """
    def initTask(**initkwargs):
        def makeIterator(**runkwargs):
            result = func(*args, **kwargs)
            yield bool(result)
        return makeIterator
    return initTask

In [20]:
# 回忆以下普通的装饰器，然后在学习以下高端的装饰器

def checktime(func, *args, **kw):
    print args, kw
    def inner(**kws):
        print kws
        print 'func begins'
        r = func(*args, **kw)
        print 'func ends'
        return r
    return inner


In [21]:
@checktime
def func():
    print 3+4

() {}


In [19]:
func()

func begins
7
func ends


In [26]:
@checktime
def func():
    print 1

() {}


In [27]:
func()

{}
func begins
1
func ends


In [42]:
from owyl import blackboard
import owyl

import time
import threading
import random


@owyl.task
def is_greater_than_10(**kwargs):
    bb = kwargs["blackboard"]
    yield bb["input"] > 10


@owyl.task
def is_between_0_and_10(**kwargs):
    bb = kwargs["blackboard"]
    yield 0 < bb["input"] < 10


@owyl.task
def wow_large_number(**kwargs):
    bb = kwargs["blackboard"]
    print "WOW, %d is a large number!" % bb["input"]
    yield True


@owyl.task
def doomed(**kwargs):
    bb = kwargs["blackboard"]
    print "%d is doomed" % bb["input"]
    yield None


@owyl.task
def count_from_1(**kwargs):
    bb = kwargs["blackboard"]
    for i in range(1, bb["input"]):
        print "count", i
        yield
    print "count", bb["input"]
    yield True


def test_bt(input_int=5):
    print "test for", input_int
    tree = owyl.selector(
            owyl.sequence(is_greater_than_10(), wow_large_number()),
            owyl.sequence(is_between_0_and_10(), count_from_1()),
#             owyl.repeat(doomed(), times=3),
            #policy=owyl.PARALLEL_SUCCESS.REQUIRE_ALL
            )
    print type(tree)
    bt = owyl.visit(tree, blackboard=blackboard.Blackboard(name="mybt", input=input_int))
    while True:
        try:
            bt.next()
        except StopIteration:
            break
    print "-"*10

In [45]:
test_bt(11)

test for 11
<type 'function'>
WOW, 11 is a large number!
----------
